In [1]:
import argparse
import os
import pickle
import pprint
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import os
from utils import *


def set_dates(df, time, mind, maxd, isIndex=False):
    if isIndex:
        newdf = df[(df.index>=mind) & (df.index<=maxd)]
    else:
        newdf = df[(df[time]>=mind) & (df[time]<=maxd)]

    return newdf


data_dir = '../../data'
country = pd.read_csv(os.path.join(data_dir, 'COVID_Cases_Restricted_Detailed_12312020.csv'), parse_dates=['cdc_report_dt', 'pos_spec_dt', 'onset_dt', 'cdc_case_earliest_dt'])


print('all cases', country.shape)
percent_missing = country.isnull().sum() * 100 / len(country)
percent_missing
print(country.shape)

#ountry = country[~country['res_state'].isin(['CT','IL','NJ'])]
print(country.shape)

min_date = pd.Timestamp('2020-04-01') - pd.Timedelta(days=6)
max_date = pd.Timestamp('2020-12-01')

country = country[country['current_status']=='Laboratory-confirmed case']


country = set_dates(country, 'cdc_case_earliest_dt', min_date, max_date)
percent_missing = country.isnull().sum() * 100 / len(country)
print(percent_missing)

print('----percent missing florida----')
percent_missing = (country[country['res_state']=='FL'].isnull().sum() * 100)/len(country[country['res_state']=='FL'])
print(percent_missing)

print(country.shape)



country['age_group'] = country['age_group'].astype('category')
country['age_group'].cat.categories
country.loc[country['age_group'].isnull(), 'age_group'] = 'Unknown'
country['age_group'].cat.rename_categories({'Unknown': '_' + 'Unknown'}, inplace=True)
country['age_group'].cat.categories

#country['age_group']
country['age_group'].cat.rename_categories({'0 - 9 Years':'0-9', 
                                       '10 - 19 Years':'10-19',
                                       '20 - 29 Years':'20-29',
                                       '30 - 39 Years':'30-39',
                                       '40 - 49 Years':'40-49', 
                                       '50 - 59 Years':'50-59',
                                       '60 - 69 Years':'60-69',
                                       '70 - 79 Years':'70-79',
                                       '80+ Years':'80+'}, inplace=True)





# hosp
country.loc[country['hosp_yn']!='Yes', 'hosp_yn'] = 0
country.loc[country['hosp_yn']=='Yes', 'hosp_yn'] = 1


# death
country.loc[country['death_yn']!='Yes', 'death_yn'] = 0
country.loc[country['death_yn']=='Yes', 'death_yn'] = 1
country = country.rename(columns={'age_group':'Age_group', 
                                    'sex': 'Gender',
                                    'hosp_yn': 'Hospitalized',
                                    'death_yn': 'Died',
                                    'current_status': 'Case'})
country = country.reset_index()


all cases (13415838, 32)
(13415838, 32)
(13415838, 32)
race_ethnicity_combined     2.245748
current_status              0.000000
cdc_case_earliest_dt        0.000000
cdc_report_dt               7.531324
sex                         0.736611
onset_dt                   50.859275
pos_spec_dt                69.286331
hosp_yn                     0.000000
icu_yn                      0.000000
death_yn                    0.000000
hc_work_yn                  2.601047
pna_yn                      0.000000
abxchest_yn                 0.070194
acuterespdistress_yn        0.000000
mechvent_yn                 0.000000
fever_yn                    0.000000
sfever_yn                   0.000000
chills_yn                   0.000000
myalgia_yn                  0.000000
runnose_yn                  0.000000
sthroat_yn                  0.000000
cough_yn                    0.000000
sob_yn                      0.000000
nauseavomit_yn              0.000000
headache_yn                 0.000000
abdom_yn            

In [3]:
country[(country['cdc_case_earliest_dt']>='2020-04-01')&(country['cdc_case_earliest_dt']<='2020-12-01')].count()


index                      10197277
race_ethnicity_combined     9968759
Case                       10197277
cdc_case_earliest_dt       10197277
cdc_report_dt               9421017
Gender                     10122283
onset_dt                    5005835
pos_spec_dt                 3099046
Hospitalized               10197277
icu_yn                     10197277
Died                       10197277
hc_work_yn                  9932707
pna_yn                     10197277
abxchest_yn                10190083
acuterespdistress_yn       10197277
mechvent_yn                10197277
fever_yn                   10197277
sfever_yn                  10197277
chills_yn                  10197277
myalgia_yn                 10197277
runnose_yn                 10197277
sthroat_yn                 10197277
cough_yn                   10197277
sob_yn                     10197277
nauseavomit_yn             10197277
headache_yn                10197277
abdom_yn                   10197277
diarrhea_yn                1

In [4]:
DATA_DIR = "../../../florida/data/"
time = 'Case_'
#print(time)
florida= pd.read_csv(os.path.join(
        DATA_DIR, 'florida_2021-01-05-15-35-01.csv'),index_col=False, parse_dates=[time, "ChartDate"])
florida["Age_group"] = florida["Age_group"].astype('category')
print(florida['Hospitalized'].value_counts())
print(florida['Hospitalized'].unique()) # 'YES'=1, else = 0
print(florida['Died'].unique()) # 'Yes'=1, nan = 0

#florida['Hospitalized'].isna()




florida.loc[florida['Hospitalized'].isna(), 'Hospitalized'] = 'Missing'
florida['Hospitalized'].value_counts()/len(florida)
print(florida.shape)
florida = set_dates(florida, 'Case_', min_date, max_date)
print(florida.shape)


florida['Age_group'] = florida['Age'].apply(lambda x: '0-9' if x < 10 
                                                           else ('10-19' if x < 20 
                                                                else('20-29'if x < 30 
                                                                    else('30-39' if x <40
                                                                        else('40-49' if x < 50
                                                                            else('50-59' if x < 60
                                                                                 else '60-69' if x<70
                                                                                    else '70-79' if x<80
                                                                                        else '80+' if not pd.isnull(x) 
                                                                                             else '_Unknown'))))))

florida['Age_group'] = florida['Age_group'].astype('category')
florida['Age_group'].cat.categories
order_cat = ['0-9',
                                             '10-19',
                                             '20-29',
                                             '30-39',
                                             '40-49',
                                             '50-59',
                                             '60-69',
                                             '70-79',
                                             '80+',
                                             '_Unknown']#[::-1]
florida['Age_group'].cat.reorder_categories(order_cat, inplace=True)
florida = florida.reset_index()


florida.loc[florida['Hospitalized']!='YES', 'Hospitalized'] = 0
florida.loc[florida['Hospitalized']=='YES', 'Hospitalized'] = 1

florida['Died'].fillna(0, inplace=True)
florida.loc[florida['Died']=='Yes', 'Died'] =1 
florida.loc[florida['Died']=='Recent', 'Died'] =1 


NO         700981
UNKNOWN    612231
YES         64783
Name: Hospitalized, dtype: int64
['NO' 'UNKNOWN' 'YES' nan]
[nan 'Yes' 'Recent']
(1392123, 15)
(1004818, 15)


In [5]:
def reorder_cat(df, var, order):
    #print(df)
    df[var] = df[var].astype('category')
    df[var].cat.reorder_categories(order, inplace=True)
    return df

def rename_cat(df, var, yn=True):
    df[var] = df[var].astype('category')
    names = {1:'Yes', 0:'No'}
    if yn:
        df[var].cat.rename_categories(names, inplace=True)
    #print(df[var].cat.categories)
   # florida['Died'].fillna(0, inplace=True)
    print(df[var].cat.categories)

    return df

florida = rename_cat(florida, 'Hospitalized')
country = rename_cat(country, 'Hospitalized')

florida = rename_cat(florida, 'Died')
country = rename_cat(country, 'Died')
florida = reorder_cat(florida, 'Hospitalized', ['Yes', 'No'])

order_age = ['0-9',
                                             '10-19',
                                             '20-29',
                                             '30-39',
                                             '40-49',
                                             '50-59',
                                             '60-69',
                                             '70-79',
                                             '80+',
                                             '_Unknown']

order_gen = ['Female', 'Male', 'Missing', 'Unknown']

Index(['No', 'Yes'], dtype='object')
Index(['No', 'Yes'], dtype='object')
Index(['No', 'Yes'], dtype='object')
Index(['No', 'Yes'], dtype='object')


In [14]:

#pd.options.display.float_format = '{:,}'.format
def fm(num, ratio):
    return "{:,} ({:.1f}%)".format(num, 100*ratio)

#def get_cat_count(s, o):
#    return s[o]
# Count
names = ['COVID-19 Cases', 'Age'] + order_age +['Gender']+order_gen+['Hospitalized', 'Yes', 'No','Died', 'Yes', 'No']
fl = []
ct = []
fl.append("{:,}".format(len(florida)))
ct.append("{:,}".format(len(country)))

def add_category(florida, country, amt, order, fl, ct):
    # for Hospitalized
    fl_h = florida[amt].value_counts()
    c_h = country[amt].value_counts()

    for o in order:
        #print(fl_h[o])
        fl.append(fm(fl_h[o], fl_h[o]/len(florida)))
        ct.append(fm(c_h[o], c_h[o]/len(country)))
    return fl, ct

def add_category_gender(florida, country, amt, order, fl, ct):
    # for Hospitalized
    fl_h = florida[amt].value_counts()
    c_h = country[amt].value_counts()

    for o in order:
        #print(fl_h[o])        
        if o!='Missing':
            fl.append(fm(fl_h[o], fl_h[o]/len(florida)))
        else:
            fl.append(fm(0, 0))
    for o in order:
            ct.append(fm(c_h[o], c_h[o]/len(country)))
    return fl, ct


fl.append('')
ct.append('')   
fl, ct = add_category(florida, country, 'Age_group', order_age, fl, ct)
fl.append('')
ct.append('')   
fl, ct = add_category_gender(florida, country, 'Gender', order_gen, fl, ct)
fl.append('')
ct.append('')     
fl, ct = add_category(florida, country, 'Hospitalized', ['Yes', 'No'], fl, ct)
fl.append('')
ct.append('')
fl, ct = add_category(florida, country, 'Died', ['Yes', 'No'], fl, ct)

    
    
    
#print(fl)
print(len(names), len(fl), len(ct))
dem = pd.DataFrame({'name':names, 'Florida':fl, 'Country':ct})
dem

dem = dem.set_index('name')
print(dem.to_latex())


23 23 23
\begin{tabular}{lll}
\toprule
{} &          Florida &             Country \\
name           &                  &                     \\
\midrule
COVID-19 Cases &        1,004,818 &          10,332,725 \\
Age            &                  &                     \\
0-9            &    37,153 (3.7\%) &      388,867 (3.8\%) \\
10-19          &    91,887 (9.1\%) &   1,045,408 (10.1\%) \\
20-29          &  192,790 (19.2\%) &   2,021,007 (19.6\%) \\
30-39          &  172,027 (17.1\%) &   1,699,260 (16.4\%) \\
40-49          &  153,717 (15.3\%) &   1,546,212 (15.0\%) \\
50-59          &  149,530 (14.9\%) &   1,485,318 (14.4\%) \\
60-69          &  102,477 (10.2\%) &   1,043,294 (10.1\%) \\
70-79          &    61,436 (6.1\%) &      583,837 (5.7\%) \\
80+            &    42,696 (4.2\%) &      453,229 (4.4\%) \\
\_Unknown       &     1,105 (0.1\%) &       66,293 (0.6\%) \\
Gender         &                  &                     \\
Female         &  517,640 (51.5\%) &   5,297,936 (51.3\%) 

In [11]:
#fl

[1004818,
 '',
 '37153 (3.7%)',
 '91887 (9.1%)',
 '192790 (19.2%)',
 '172027 (17.1%)',
 '153717 (15.3%)',
 '149530 (14.9%)',
 '102477 (10.2%)',
 '61436 (6.1%)',
 '42696 (4.2%)',
 '1105 (0.1%)',
 '',
 '517640 (51.5%)',
 '482227 (48.0%)',
 '0 (0.0%)',
 '4951 (0.5%)',
 '',
 '56673 (5.6%)',
 '948145 (94.4%)',
 '',
 '21028 (2.1%)',
 '983790 (97.9%)']

In [7]:
pd.options.display.float_format = '{:,}'.format


In [8]:
print(dem.to_latex())

\begin{tabular}{lll}
\toprule
{} &         Florida &           Country \\
name           &                 &                   \\
\midrule
COVID-19 Cases &         1004818 &          10332725 \\
Age            &                 &                   \\
0-9            &    37153 (3.7\%) &     388867 (3.8\%) \\
10-19          &    91887 (9.1\%) &   1045408 (10.1\%) \\
20-29          &  192790 (19.2\%) &   2021007 (19.6\%) \\
30-39          &  172027 (17.1\%) &   1699260 (16.4\%) \\
40-49          &  153717 (15.3\%) &   1546212 (15.0\%) \\
50-59          &  149530 (14.9\%) &   1485318 (14.4\%) \\
60-69          &  102477 (10.2\%) &   1043294 (10.1\%) \\
70-79          &    61436 (6.1\%) &     583837 (5.7\%) \\
80+            &    42696 (4.2\%) &     453229 (4.4\%) \\
\_Unknown       &     1105 (0.1\%) &      66293 (0.6\%) \\
Gender         &                 &                   \\
Female         &  517640 (51.5\%) &   5297936 (51.3\%) \\
Male           &  482227 (48.0\%) &   4861508 (47.0\%)